# Calidad de los datos de una serie de tiempo

Los decibeles expresan una relación de poder, no una cantidad. Dicen cuántas veces más (dB positivo) o menos (dB negativo) pero no cuánto en términos absolutos. Los decibeles son logarítmicos, no lineales. Por ejemplo, 20 dB no es el doble de la relación de potencia de 10 dB.

La ecuación para decibelios es:

$A = 10 * log10 (P2 / P1) (dB)$

donde P1 es la potencia que se mide y P1 es la referencia con la que se compara P2.
Para convertir de la medida de decibelios a la relación de potencia:

$P2 / P1 = 10^{(A / 10)}$

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from lofTS.lofTS import lofTS

In [2]:
from pandas import read_csv
from pandas import DataFrame
import time, json
import random

from sklearn.neighbors import LocalOutlierFactor
from scipy import signal, fft
from statsmodels.tsa.stattools import adfuller, acf, pacf
import statsmodels.graphics as smg

%matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.font_manager

import bokeh
from bokeh.plotting import gridplot, figure, output_file, output_notebook, show
from bokeh.models import Legend

#Libreria para graficar en 3D
# import ipyvolume as ipv

from datetime import datetime
# from __future__ import print_function

from PyAstronomy import pyasl
from pylab import *
# Biblioteca para calcular los exponentes de Lyapunov
import nolds 
import datetime

## Graficar 2d

In [3]:
def grafica2D(x1,y1,x2=[],y2=[],titulo="Grafica",title1="Serie 1",title2="Serie 2",tipo=1):    
    p = bokeh.plotting.figure(title=titulo,plot_width=500, plot_height=300)
    output_notebook()
    legend_it = []

    if tipo == 1:
        c = p.line(x1, y1, color="black")
        #p.circle(x1,y1, color="red")
        legend_it.append((title1, [c]))
        if(len(y2)>0):
            c = p.line(x2, y2, color="red")
            legend_it.append((title2, [c]))
    else:
        p.line(x1,y1, color="black")
        if(len(y2)>0):
            p.circle(x2,y2, color="red")
    legend = Legend(items=legend_it, location=(0, 10))
    legend.click_policy="mute"
    p.add_layout(legend, 'above')
    
    bokeh.plotting.show(p, browser=None, new='tab', notebook_handle=False, notebook_url='localhost:8888')

## Valores faltantes

In [4]:
def valoresFaltantes(data):
    tam = len(data)
    num_nans = round(round(data.isnull().sum())/float(tam)*100) 
#     print("Porcentaje de valores faltantes",num_nans,"tamano de la serie de tiempo",tam)
    return num_nans

## Outliers

In [10]:
def obtenerOutliers(datos,k=20,showPlot=0):
    """obtenerOutliers es una funcion que encuentra los valores atipicos globales con el algoritmo de la regla 3 sigma
    y encuentra los valores atipicos locales con el algoritmo de lof 
    
    Entradas: 
        datos.- Es un arreglo de numpy
        k.- Numero de vecinos
        showPlot.- Por defecto el valor 0 indica que no se muestre la grafica si es 1 se muestra"""
    np.random.seed(42)
#     print("incio LocalOutlierFactor")
    clf = lofTS(n_neighbors=k,metric='euclidean'
                             ,algorithm = 'ball_tree',leaf_size = 5, n_jobs=-1)
#     print("fin LocalOutlierFactor \nclf",clf)
    #print("DATA/n",datos['col1'][0:130])
    datos2 = datos
    datos = datos.dropna()
    y_pred = clf.fit_predict(datos)
    #print("\n\ny_pred")
    posiciones = []
    outliers = []
    inliers = []
    dfOut = pd.DataFrame()
    dfIn = pd.DataFrame()
    ###
    mu = np.mean(datos['col1'])
    sigma = np.std(datos['col1'])
            
    lim_sup = mu+(3*sigma)
    lim_inf = mu-(3*sigma)
    print("limite supe",lim_sup,"\nlim_inf",lim_inf)
    indi_up = np.ravel(np.where(datos['col1'] > lim_sup))
    indi_low = np.ravel(np.where(datos['col1'] < lim_inf))
    print("indices",indi_up,"indi_low",indi_low)          
    total_outliers = len(indi_up) + len(indi_low)
    print("total outliers 3sigma",total_outliers)
    ###

    if(np.shape(datos)[1] == 2):
        outliers = [datos2['col0'][np.ravel(np.where(y_pred==-1))],
                    datos2['col1'][np.ravel(np.where(y_pred==-1))]]
        inliers = [datos2['col0'][np.ravel(np.where(y_pred==1))],
                    datos2['col1'][np.ravel(np.where(y_pred==1))]]
        dfOut = pd.DataFrame({'col0':outliers[0],'col1':outliers[1]})
        dfIn = pd.DataFrame({'col0':inliers[0],'col1':inliers[1]})
        print("dfOut\n\n",len(dfOut),"\n",dfOut)
        print("dfIn\n\n",len(dfIn),"\n",dfIn)
        superin = np.concatenate((indi_up, indi_low), axis=0)
        superin = np.sort(superin)
        superin = superin[::-1]
        print("superin",superin)
        #print("indi_up size",len(indi_up),"ind_low size",len(indi_low),"append ind_sup.ind_inf",len(superin))
        for i in superin:
            #print("\nOUTLIERS\ni",outliers[0].shape,"\nindices\n",indi_up)
            if not (i in outliers[0]):
                dfOut = dfOut.append({'col0': i, 'col1':datos2['col1'][i]}, ignore_index=True)
                dfIn = dfIn.drop(dfIn['col0'][i])
                #print("si entro son diferentes")
            #print("dfOut agregado\n\n",dfOut)
        #print("tam despues dfOut",len(dfOut))
        #print("tam despues dfIn",len(dfIn))
        if showPlot:
            grafica2D(dfIn['col0'],dfIn['col1'],dfOut['col0'],dfOut['col1'],
                      titulo = "Inliers vs Outliers", tipo = 2)
#         ipv.quickscatter(inliers[0], inliers[1], color = "blue", size=0.1, marker="sphere")
#         ipv.scatter(outliers[0], outliers[1], color = "red", size=0.5, marker="sphere")
#         ipv.show()
    elif(np.shape(datos)[1] == 3):
        outliers = [datos['col0'][np.ravel(np.where(y_pred==-1))],
                    datos['col1'][np.ravel(np.where(y_pred==-1))],
                    datos['col2'][np.ravel(np.where(y_pred==-1))]]
        inliers = [datos['col0'][np.ravel(np.where(y_pred==1))],
                    datos['col1'][np.ravel(np.where(y_pred==1))],
                    datos['col2'][np.ravel(np.where(y_pred==1))]]
        if showPlot:
            ipv.quickscatter(inliers[0], inliers[1], inliers[2], color = "blue", size=0.1, marker="sphere")
            ipv.scatter(outliers[0], outliers[1], outliers[2], color = "red", size=0.5, marker="sphere")
            ipv.show()

#     print("inicia negative_outlier_factor_")
    Z = -clf.negative_outlier_factor_
#     print("termina negative_outlier_factor_")
#     print("inicia _decision_function")
#     fD = clf._decision_function(datos)
#     print("termina _decision_function")
    if showPlot:
        print("Datos analizados:",len(y_pred),"\nInlliers:",len(dfIn),
              "\nOutliers",len(dfOut))
        print("Media",np.mean(Z),"\nDst",np.std(Z),
              "\nDif",np.abs(Z[np.argmax(Z)]-np.mean(Z)),
              "\nDonde estan",outliers,
              "\nn_neighbors_",clf.n_neighbors_,
              "\nIndice LOF",Z)
    return dfIn,dfOut,posiciones

def calcularOutliers(nombre,original=0,faltantes=0,outliers=0,SNR_dB=0):
    ts = pd.DataFrame()
    tam = 0
    if original == 1:
        filename = "st originales/"+nombre+".dat"
        ts_aux = pd.read_csv(filename)
        ts_aux = ts_aux.dropna()
        tam = len(ts_aux)
        if tam > 20000:
            ts_aux = ts_aux[0:20000]
        ts = pd.DataFrame({'col0': np.arange(len(ts_aux)),'col1': ts_aux['col1']})
    else:
        filename = "st perturbadas/"+nombre+"/"+nombre+"_"+str(faltantes)+"_"+str(outliers)+"_"+str(SNR_dB)+".csv"
        ts = pd.read_csv(filename)
        ts = ts.dropna()
        tam = len(ts)
        if tam > 20000:
            ts = ts[0:20000]
#         print("TSSSS\n",ts)
#     print(ts)
    Inliers,Outliers,Pos = obtenerOutliers(ts,k=5,showPlot=1)
#     print(len(Outliers[1]))
    tam = len(ts)
    totalOutliers = round(len(Outliers)/float(tam)*100)
#     print("totalOutliers",totalOutliers)
    return totalOutliers
# calcularOutliers("duffing_oscillator",original=1)

## SNR

In [11]:
def obtener_SNR(xn,x=[],t=[],filename='SNR.csv',showPlot=0):
    """
    Parameters:
           xn.- Signal + noise
           x.- Original signal
           t.- time
           filename.- donde guardar los SNR obtenidos
           showPlot.- muestra las graficas con 1 y con 0 no
    Return:
           SNR.- Signal to noise ratio 
    """
    # Estimate noise using robust estimate
    beq = pyasl.BSEqSamp()
    # Define order of approximation (use larger values such as 2,3, or 4 for
    # faster varying or less well sampled data sets; also 0 is a valid order)
    N = 2
    # Define 'jump parameter' (use larger values such as 2,3, or 4 if correlation
    # between adjacent data point is suspected)
    j = 1
    nstd, nstdstd = beq.betaSigma(xn['col1'], N, j, returnMAD=True)
    # Obteniendo la varianza apartir de la desviacion estandar
    var_noise = np.power(nstd,2)
    # Ma va a contener la senal
    Ma = []
    # Res contiene los residuos los cuales se consideran como el ruido
    Res = []
    SNRcalculado = 0
    for window in range(2,20):
#         print("Ancho de ventana = ",window)
        # Moving Average Medias Moviles
        Ma = xn['col1'].rolling(window=window,center=True).mean()
        # Residuos de la Media movil
        Res = xn['col1'] - Ma
        # Quitando los 
        Res = Res.dropna()
        # Obteniendo la varianza del ruido
        varianza_Noise_sin = np.var(Res)      
#         restd = np.std(Ma)
        # Obteniendo las frecuencias y el espectro de potencia o densidad espectral de potencia
        fsenal, Pxx_den_senal = signal.periodogram(Ma[~np.isnan(Ma)])
        fnoise, Pxx_den_noise = signal.periodogram(Res[~np.isnan(Res)])
        # Quitando nans de la densidad espectral de potencia tanto para la senal como para el ruido
        Pxx_den_senal = Pxx_den_senal[~np.isnan(Pxx_den_senal)]
        Pxx_den_noise = Pxx_den_noise[~np.isnan(Pxx_den_noise)]

        #Si la diferencia entre la varianza de pyastronomy y la varianza del ruido extraido es menor a 1e-3
        #Podemos decir que se ha encontrado el SNR o que no contiene SNR
        if np.abs(var_noise - varianza_Noise_sin) < 1e-3:
            # Si la ventana es de 2 quiere decir que la serie no contiene ruido
            if window == 2:
                print("No contiene ruido")
                filename.write(",,No hay ruido\n")
                SNRcalculado = "No hay ruido"
                break
            # Esta x se utilizo para las pruebas
            if len(x) > 0:
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                filename.write(str(SNRcalculado)+'\n')
                if showPlot:
                    grafica2D(t,Ma,t,x,
                             title1='Ruido reducido',title2='Original')
                    grafica2D(t,Ma,t,xn['col1'],
                             title1='Ruido reducido',title2='Ruido')
            else:
                # Se calcula el SNR = Psignal/Pnoise donde P es la potencia media https://en.wikipedia.org/wiki/Signal-to-noise_ratio
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
                filename.write(str(SNRcalculado)+'\n')
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    grafica2D(t,Ma,t,xn['col1'],
                             title1='Noise reduced',title2='Noise signal')
            break
        if window == 19:
            SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
            filename.write(str(SNRcalculado)+',,,Max\n')
    return SNRcalculado


#Funcion para graficar el periodograma
# import matplotlib.pyplot as plt
def plot_periodogram(df_x,title,legend,showPlot=0):
    #Obtenemos la frecuencia y densidad espectral de potencia o espectro de potencia.
    f, Pxx_den = signal.periodogram(df_x['col1'].T)
    Pxx_den = np.ravel(Pxx_den)
    Pxx_den[0] = np.nan
    #print(Pxx_den[0:10])
    if showPlot:
        p = bokeh.plotting.figure(title=title, y_axis_type="log",
               background_fill_color="#fafafa")
        p.line(x=f, y=np.ravel(Pxx_den), legend=legend, line_color="blue")
        bokeh.plotting.show(p)
    return f, Pxx_den

def variacion(Pxx_den):
    # Esta funcion recibe el espectro de potencia
    # El indice es el del elemento mas grande
    indice = np.argmax(Pxx_den[~np.isnan(Pxx_den)])
    # Si el indice se encuentra delante del 10% de la posicion quiere decir que no se puede calcular el SNR
    if indice > len(Pxx_den)*0.1:
        return False
    else:
        return True

def plotseries(tipo,path="",ts=[]):
    # Funcion para plotear las series
    if tipo == 0:
        grafica2D(np.arange(len(ts)),ts,title1='TS')
    else:
        ts = pd.read_csv(path)
#         ts = ts.dropna()
        grafica2D(np.arange(len(ts)),ts['col1'],title1='TS')

In [12]:
def SNRTsPruebas(nombre,prueba=0,SNR_dB=-1,showPlot=0):
    path = "st con ruido/"+nombre+str(SNR_dB)
    numeroDeSeries = 1
    if prueba:
        numeroDeSeries = 10
    SNRObtenido = 0
    if SNR_dB == -1:
        print("entra",os.path.exists)
        if not os.path.exists:
            print("entra")
            os.mkdir(path)
        numeroDeSeries = 1
    filename = path+"/snr "+nombre+str(SNR_dB)+".csv"
    csv_file = open(filename, "w")  
    columnTitleRow = "SNR detectado, SNR no detectado, No contiene SNR, Prueba con un SNR = "+str(SNR_dB)+'\n'
    csv_file.write(columnTitleRow)
    tiempoGlobal = time.time()
    for i in range(0,numeroDeSeries):
        tiempoInicioTarea = time.time()
        print("i",i)
        if SNR_dB != -1:
            ts = pd.read_csv("st con ruido/"+nombre+str(SNR_dB)+"/ts "+str(i)+".csv")
        else:
            #Read the file 
            ts = pd.read_csv("st originales/"+nombre+".dat")
            ts = ts[0:20000]
            ts = ts.dropna()
#         grafica2D(np.arange(len(ts)),ts['col1'],title1=nombre+" SNR "+str(SNR_dB))    
        f_ts, Pxx_den_ts = plot_periodogram(ts,nombre,nombre,showPlot=showPlot)
        varia = variacion(Pxx_den_ts)
        if varia:
            #Obtener el SNR
            SNRObtenido = obtener_SNR(ts,t=np.arange(len(ts)),filename=csv_file,showPlot=showPlot)
        else:
            print("Varia rapidamente")
            SNRObtenido = "No se puede calcular SNR"
            csv_file.write(",No se puede calcular SNR,\n")
        tiempoFinalTarea = time.time()
        print("Tiempo total ",tiempoFinalTarea-tiempoGlobal,"Tiempo tarea ",tiempoFinalTarea-tiempoInicioTarea)
    columnTitleRow = "=PROMEDIO(A2:A11)\n"
    csv_file.write(columnTitleRow)
    csv_file.close()
    return SNRObtenido

In [13]:
def SNRTs(nombre,faltantes=0,outliers=0,SNR_dB=0,showPlot=0):
    tiempoGlobal = time.time()
    path = "st perturbadas/"+nombre+"/"+nombre+"_"+str(faltantes)+"_"+str(outliers)+"_"+str(SNR_dB)+".csv"
    SNRObtenido = 0
    ts = pd.read_csv(path)
    if len(ts) > 20000:
        ts = ts[0:20000]
    ts = ts.dropna()
    
    filename = "snr/"+nombre+str(SNR_dB)+".csv"
    csv_file = open(filename, "w")  
    columnTitleRow = "SNR detectado, SNR no detectado, No contiene SNR, Prueba con un SNR = "+str(SNR_dB)+'\n'
    csv_file.write(columnTitleRow)
    
#     ts = ts.dropna()
#     grafica2D(np.arange(len(ts)),ts['col1'],title1=nombre+" SNR "+str(SNR_dB))    
    f_ts, Pxx_den_ts = plot_periodogram(ts,nombre,nombre,showPlot=showPlot)
    varia = variacion(Pxx_den_ts)
    print("varia",varia)
    if varia:
        #Obtener el SNR
        SNRObtenido = obtener_SNR(ts,t=np.arange(len(ts)),filename=csv_file,showPlot=showPlot)
    else:
#         print("Varia rapidamente")
        SNRObtenido = "No se puede calcular SNR"
    csv_file.close()
    tiempoFinalTarea = time.time()
    print("Tiempo total SNRTs",tiempoFinalTarea-tiempoGlobal)
    return SNRObtenido

In [17]:
# print(calcularOutliers("lorenz",original=0,faltantes=0,outliers=5,SNR_dB=0))
# plotseries(1,path="st perturbadas/halvorsen_attractor/halvorsen_attractor_10_25_0.csv")
windAristeoM10m = pd.read_csv('st perturbadas/chen_system/chen_system_0_10_0.csv')

obtenerOutliers(windAristeoM10m,k=5,showPlot=1)
# plotseries(1,path="st originales/henon.dat")
# print(SNRTsPruebas("henon",prueba=0,SNR_dB=-1,showPlot=1))
# SNRTs("henon",0,0,0,showPlot=1)

limite supe 34.68908351066535 
lim_inf -34.690858068315414
indices [   85    90    92   103   964   965   972  1002  1005  1010  1031  1038
  1041  1046  1066  1903  1905  1909  1926  1947  1948  1949  1958  1963
  1969  1971  1978  1989  2010  3085  3090  3104  3129  3135  4312  4315
  4319  4324  4338  4340  4341  4350  4355  5211  5237  5769  5787  5788
  5800  5814  5824  5827  5833  5843  5861  5863  5866  6782  6786  6801
  6807  6814  7843  8532  8547  8550  8559  8560  8568  8571  8575  8579
  8596  8598  8600  8614  8626  8860  9339  9341  9372  9391  9395  9401
  9420  9422  9920  9922  9954  9969  9972  9974  9975  9978  9984  9996
  9999 10003 10518 10523 10525 10526 10546 10555 10558 10565 10579 10581
 10591 10594 11091 11116 11126 11139 11141 11146 11153 11158 11738 11739
 11748 11753 11763 11775 11778 11783 11786 11792 11807 11826 13517 13526
 13534 13550 13553 13558 13580 13585 13609 13613 13617 14530 14537 14542
 14553 14573 14575 15417 15419 15428 15436 15445 15455 15

Loading BokehJS ...

Datos analizados: 20000 
Inlliers: 19084 
Outliers 916
Media 1.1168121730654348 
Dst 0.5170428689817556 
Dif 7.1034364063748345 
Donde estan [15          15
32          32
40          40
63          63
103        103
121        121
142        142
147        147
162        162
181        181
287        287
301        301
317        317
382        382
539        539
577        577
586        586
634        634
643        643
652        652
658        658
667        667
688        688
698        698
705        705
713        713
757        757
773        773
783        783
794        794
         ...  
19288    19288
19320    19320
19329    19329
19345    19345
19355    19355
19391    19391
19412    19412
19434    19434
19453    19453
19475    19475
19485    19485
19542    19542
19564    19564
19574    19574
19586    19586
19628    19628
19645    19645
19668    19668
19672    19672
19677    19677
19683    19683
19703    19703
19711    19711
19776    19776
19786    19786
19807    19807
198

(        col0       col1
 0          0 -10.000000
 1          1  -9.650000
 2          2  -9.296850
 3          3  -8.940806
 4          4  -8.582121
 5          5  -8.221047
 6          6  -7.857836
 7          7  -7.492733
 8          8  -7.125982
 9          9  -6.757825
 10        10  -6.388499
 11        11  -6.018235
 12        12  -5.647264
 13        13  -5.275809
 14        14  -4.904091
 16        16  -4.160720
 17        17  -3.789482
 18        18  -3.418810
 19        19  -3.048900
 20        20  -2.679941
 21        21  -2.312118
 22        22  -1.945608
 23        23  -1.580588
 24        24  -1.217224
 25        25  -0.855681
 26        26  -0.496116
 27        27  -0.138684
 28        28   0.216467
 29        29   1.707581
 30        30   0.919356
 ...      ...        ...
 19970  19970   5.837939
 19971  19971   5.897547
 19972  19972   5.957352
 19973  19973   6.017361
 19974  19974   6.077581
 19975  19975  18.414052
 19976  19976  18.596033
 19977  19977   6.259568


## Caos

In [ ]:
def calcularCaosPrueba(nombre,SNR_dB=-1):
    # Funcion para calcular el Caos usando la biblioteca nolds y el algoritmo de Rosenstein
#     tiempoInicial = time.time()
    filename = ""
    # Revisando si la st está contaminada con ruido o es la original
    if SNR_dB != -1:
        filename = "st con ruido/"+nombre+str(SNR_dB)+"/ts 0"+".csv"
    else:
        filename = "st originales/"+nombre+".dat"
    # Se carga la ts
    ts = pd.read_csv(filename)
    tamano = len(ts)
    # Se limita a que a lo maximo el tamano de la ts sea 20000 
    if tamano > 20000:
        tamano = 20000
    ts = ts[0:tamano]
#     print("TS\n\n",ts['col1'].shape)
    # Se calcula el exponente
    expo = nolds.lyap_r(np.ravel(ts['col1']))
    #expo = max(nolds.lyap_e(ts['col1']))
#     print("Tiempo total ",time.time()-tiempoInicial)
    return expo

def calcularCaos(nombre,faltantes=0,outliers=0,SNR_dB=0,showPlot=0):
    tiempoGlobal = time.time()
    path = "st perturbadas/"+nombre+"/"+nombre+"_"+str(faltantes)+"_"+str(outliers)+"_"+str(SNR_dB)+".csv"
    ts = pd.read_csv(path)
    if len(ts) > 20000:
        ts = ts[0:20000]
    ts = ts.dropna()
    expo = nolds.lyap_r(np.ravel(ts['col1']))
    #expo = max(nolds.lyap_e(ts['col1']))
#     print("Tiempo total ",time.time()-tiempoInicial)
    return expo

## Obtener caracteristicas

In [ ]:
def obtenerCaracteristicas(nombreTs,showPlot=0):
    niveles = [0,5,10]
    tam = 0
    faltantes = 0
    outliers = 0
    snr = 0
    caos = 0
    
    tiempoInicial = time.time()
    #####
    
    filename = "caracteristicas/"+nombreTs+str(datetime.datetime.now())+".csv"
    csv_file = open(filename, "w")
    columnTitleRow = "Tamano ts,Va_Fa,Out,SNR_In,Valores Faltantes,Outliers,SNR,Caos\n"
    csv_file.write(columnTitleRow)    
    for i in niveles:
        for j in niveles:
            for k in niveles:
                print(i,j,k)
                # filename = "st con ruido/seno5/ts 0.csv"
                filename = "st perturbadas/"+nombreTs+"/"+nombreTs+"_"+str(i)+"_"+str(j)+"_"+str(k)+".csv"
                ts_aux = pd.read_csv(filename)
                ts_aux = ts_aux['col1']
                tam = len(ts_aux)
                if tam > 20000:
                    tam = 20000
                    ts_aux = ts_aux[0:20000]
                # Obtener los valores faltantes
                #if i != 0:
                faltantes = valoresFaltantes(ts_aux)
                # Obtener los outliers
                #if j != 0:
                outliers = calcularOutliers(nombreTs,original=0,faltantes=i,outliers=j,SNR_dB=k)
                #if k != 0:                    
                snr = SNRTs(nombreTs,i,j,k,showPlot=0)
                caos = calcularCaos(nombreTs,i,j,k,showPlot=0)
                csv_file.write(str(tam)+","+str(i)+","+str(j)+","+str(k)+","+str(faltantes)+","+str(outliers)+","+str(snr)+","+str(caos)+'\n')
                # Ruta donde se va a guardar la ts
    csv_file.close()

    print("Tiempo total ",time.time()-tiempoInicial)

In [ ]:
SC = ["seno","henon","lorenz","rossler","chen_system","duffing_oscillator","halvorsen_attractor",
      "rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
obtenerCaracteristicas(SC[1],showPlot=0)

### Caos

In [ ]:
#ACT_attractor,"chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"
SC = ["seno","henon","lorenz","rossler","ACT_attractor","chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
# SC = ["henon"]
SNR_pasos = np.arange(4,5)
print(SNR_pasos)
for i in SC:
#     filename = "caos/"+i+" caos"+str(tamano)+".csv"
#     csv_file = open(filename, "w")
#     columnTitleRow = "Caos"+str(tamano)+"\n"
#     csv_file.write(columnTitleRow)
    caos = 0
    filename = "caos/"+i+" caos.csv"
    csv_file = open(filename, "w")
    for j in SNR_pasos:
        if j == 4:
            caos = calcularCaos(i)
            csv_file.write("0,"+str(caos)+'\n')
        else:
            caos = calcularCaos(i,j)
            csv_file.write(str(j)+","+str(caos)+'\n')
#         print(i,j,caos)
    csv_file.close()

### Periodograma

Cualquier serie de tiempo puede expresarse como una combinación de ondas de coseno (o seno) con diferentes períodos (cuánto tiempo lleva completar un ciclo completo) y amplitudes (valor máximo / mínimo durante el ciclo). Este hecho se puede utilizar para examinar el comportamiento periódico (cíclico) en una serie de tiempo.

Un periodograma se utiliza para identificar los períodos dominantes (o frecuencias) de una serie de tiempo. Esta puede ser una herramienta útil para identificar el comportamiento cíclico dominante en una serie, especialmente cuando los ciclos no están relacionados con la estacionalidad mensual o trimestral más común.


En el área de series de tiempo llamada análisis espectral, vemos una serie de tiempo como una suma de ondas de coseno con amplitudes y frecuencias variables. Uno de los objetivos de un análisis es identificar las frecuencias (o períodos) importantes en la serie observada. Una herramienta de partida para hacer esto es el periodograma. El periodograma grafica una medida de la importancia relativa de los posibles valores de frecuencia que podrían explicar el patrón de oscilación de los datos observados.

Supongamos que hemos observado datos en n puntos temporales distintos y, por conveniencia, suponemos que n es par. Nuestro objetivo es identificar frecuencias importantes en los datos. Para continuar con la investigación, consideramos el conjunto de posibles frecuencias $ω_{j} = j/n$ para $j=1, 2, ..., n/2$. Estas se llaman las frecuencias armónicas.

Representaremos las series de tiempo como

$x_{t} = \sum^{n/2}_{j=1}[β_{1} (j_{n}) cos(2πω_{j}t) + β_{2}(j_{n}) sen (2πω_{j}t)]$

Esta es una suma de las funciones de seno y coseno en las frecuencias armónicas. La forma de la ecuación proviene de la identidad dada anteriormente.

Piense en los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$ como parámetros de regresión. Luego hay un total de n parámetros porque permitimos que j se mueva de $1$ a $n/2$. Esto significa que tenemos n puntos de datos y n parámetros, por lo que el ajuste de este modelo de regresión será exacto.

El primer paso en la creación del periodograma es la estimación de los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$. Realmente no es necesario llevar a cabo esta regresión para estimar los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$. En su lugar, se utiliza la Transformada Rápida de Fourier (FFT).

Una vez estimados los parámetros, definimos


https://newonlinecourses.science.psu.edu/stat510/node/71/

In [ ]:
from psr import psr
from psr import methods
import pandas as pd

In [ ]:
def crearbd(nombreTs):
    niveles = [0,5,10,15,20,25]
    filename = "parametrosbd/"+nombreTs+str(datetime.datetime.now())+".csv"
    csv_file = open(filename, "w")
    columnTitleRow = "Tamano ts,Va_Fa,Out,SNR_In,m,tau,tiempo,\n"
    csv_file.write(columnTitleRow)    
    csv_file.close()    
    tiempoGlobal = time.time()
    for i in niveles:
        for j in niveles:
            for k in niveles:
                tiempoPorSerie = time.time()
                with open(filename, 'a') as csv_file:
                    writer = csv.writer(csv_file)
    #                 nameTs = "simplest_linear_flow"
                    filename = "st perturbadas/"+nameTs+"/"+nameTs+"_"+str(i)+"_"+str(j)+"_"+str(k)+".csv"
                    ts = pd.read_csv(filename)
                    ts = ts['col1']
                    dm = methods.DeterministicMethod(ts)
                    m, tau = dm.deterministic_method(2, 7)
                    p = psr.PSR(ts = np.array(ts), m= m, tau=tau)
                    db = p.get_database(path="st perturbadas/"+nameTs+"/"+nameTs+"_"+str(i)+"_"+str(j)+"_"+str(k)+'_bd.csv')
                    row = [len(ts),i,j,k,m,tau]
                    writer.writerow(row)
#                     csv_file.write(str(tam)+","+str(i)+","+str(j)+","+str(k)+","+str(faltantes)+","+str(outliers)+","+str(snr)+","+str(caos)+'\n')
                    # Ruta donde se va a guardar la ts
                csv_file.close()
                print("Tiempo por serie",time.time()-tiempoPorSerie)
    print("termino tiempo total",time.time()-tiempoGlobal)
                    # chen_system                  m = 6, tau=1, 38.175167083740234
                    # duffing_oscillator           m = 6, tau=1, 29.691254377365112s
                    # halvorsen_attractor          m = 6, tau=1, 31.72951078414917
                    # henon                        m = 6, tau=1, 30.108326196670532
                    # lorenz                       m = 6, tau=17, 37.301207065582275
                    # rossler                      m = 6, tau=1, 34.7912802696228
                    # rucklidge_attractor          m = 6, tau=1, 31.89549160003662
                    # seno                         m = 6, tau=1, 0.3272838592529297
                    # shawn_van_der_pol_oscillator m = 6, tau=1, 30.14395260810852
                    # simplest_cubic_flow          m = 6, tau=6, 32.612558364868164
                    # simplest_linear_flow         m = 6, tau=1, 29.60354256629944

In [ ]:
def ob_SNR(xn,x=[],t=[],filename='SNR.csv',showPlot=0):
    """
    Parameters:
           xn.- Signal + noise
           x.- Original signal
           t.- time
           filename.- donde guardar los SNR obtenidos
           showPlot.- muestra las graficas con 1 y con 0 no
    Return:
           SNR.- Signal to noise ratio 
    """
    # Estimate noise using robust estimate
    beq = pyasl.BSEqSamp()
    # Define order of approximation (use larger values such as 2,3, or 4 for
    # faster varying or less well sampled data sets; also 0 is a valid order)
    N = 2
    # Define 'jump parameter' (use larger values such as 2,3, or 4 if correlation
    # between adjacent data point is suspected)
    j = 1
    nstd, nstdstd = beq.betaSigma(xn['col1'], N, j, returnMAD=True)
    # Obteniendo la varianza apartir de la desviacion estandar
    var_noise = np.power(nstd,2)
    # Ma va a contener la senal
    Ma = []
    # Res contiene los residuos los cuales se consideran como el ruido
    Res = []
    SNRcalculado = 0
    if len(x) == 0 and len(t) == 0:
        x = xn['col1']
        t = np.arange(len(xn))
    for window in range(2,20):
#         print("Ancho de ventana = ",window)
        # Moving Average Medias Moviles
        Ma = xn['col1'].rolling(window=window,center=True).mean()
        # Residuos de la Media movil
        Res = xn['col1'] - Ma
        # Quitando los 
        Res = Res.dropna()
        # Obteniendo la varianza del ruido
        varianza_Noise_sin = np.var(Res)      
#         restd = np.std(Ma)
        # Obteniendo las frecuencias y el espectro de potencia o densidad espectral de potencia
        fsenal, Pxx_den_senal = signal.periodogram(Ma[~np.isnan(Ma)])
        fnoise, Pxx_den_noise = signal.periodogram(Res[~np.isnan(Res)])
        # Quitando nans de la densidad espectral de potencia tanto para la senal como para el ruido
        Pxx_den_senal = Pxx_den_senal[~np.isnan(Pxx_den_senal)]
        Pxx_den_noise = Pxx_den_noise[~np.isnan(Pxx_den_noise)]

        #Si la diferencia entre la varianza de pyastronomy y la varianza del ruido extraido es menor a 1e-3
        #Podemos decir que se ha encontrado el SNR o que no contiene SNR
        if np.abs(var_noise - varianza_Noise_sin) < 1e-3:
            # Si la ventana es de 2 quiere decir que la serie no contiene ruido
            if window == 2:
                print("No contiene ruido")
                SNRcalculado = "No hay ruido"
                break
            # Esta x se utilizo para las pruebas
            if len(x) > 0:
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    print("ventana =",window)
                    grafica2D(t,x,t,Ma,
                             title1='Original',title2='Ruido reducido')
                    grafica2D(t,xn['col1'],t,Ma,
                             title1='Ruido',title2='Ruido reducido')
            else:
                # Se calcula el SNR = Psignal/Pnoise donde P es la potencia media https://en.wikipedia.org/wiki/Signal-to-noise_ratio
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    print("ventana =",window)
                    grafica2D(t,xn['col1'],t,Ma,
                             title1='St. con ruido',title2='St. con ruido reducido')
            break
        if window == 19:
            SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
            if showPlot:
                print("ventana =",window)
                grafica2D(t,x,t,Ma,
                         title1='St. con ruido',title2='St. con ruido reducido')
    return SNRcalculado

In [ ]:
ts = pd.read_csv('st perturbadas/rossler/rossler_0_0_25.csv')
#obtenerOutliers(windAristeoM10m,k=5,showPlot=1)
ob_SNR(ts,showPlot=1)

In [1]:
# coding: utf-8

from os import listdir
from os.path import isfile, join
from ppd.ppd import preprocessingTS

import pandas as pd

# Parquet support for Pandas
import pyarrow
import time

import json
import numpy as np

Load ppd


In [12]:
nane = "ABT04015"
Parquetts = pd.read_parquet("/Users/victormanueltellez/Documents/MIRD/Segundo_preproceso_nuevos_modelos__offline/Data/parquet/hourly/"+nane+".parquet")

In [13]:
Parquetts.to_csv("/Users/victormanueltellez/Documents/MIRD/"+nane+".csv")